# Data and Configuration

This notebook provides examples of data pipelines, data space building, and configuration managment, in addition to examples of model components being used (see section below).
The following examples are provided:
* Baseline parameter confiugration
* Data space

In [ ]:
from giga.app.config import ConfigClient, get_config

# we can specify the default configuration to load data from a local workspace
workspace = '../sample_workspace' # the workspace where data for each country of interest can be found
country = 'sample' # the identifier for the country of interest (rwanda, and brazil are the other options)
default_config = ConfigClient(get_config([f"data={country}", f"data.workspace={workspace}"]))

# Component Drivers

This notebook provides examples of component drivers that show how to configure, and run a model component.
The following are shown:

* Fiber Model
* Satellite Model
* Cellular Model


## Data Space

## Fiber Component Model

In [ ]:
from giga.app.config import ConfigClient, get_config

workspace = "../sample_workspace"
global_config = ConfigClient(get_config(["data=sample", f"data.workspace={workspace}"]))

In [ ]:
from giga.models.components.fiber_cost_model import FiberCostModel
from giga.schemas.conf.models import FiberTechnologyCostConf
from giga.data.space.model_data_space import ModelDataSpace


# Specify all the configurations, there are parsers that can help with this in the library
config = FiberTechnologyCostConf(
            capex={
                "cost_per_km": 7_500, # USD
                "economies_of_scale": True,
            },
            opex={
                "cost_per_km": 100, # USD
                "annual_bandwidth_cost_per_mbps": 10, # in USD
            },
            constraints={
                "maximum_connection_length": 20_000, # in meters
                "required_power": 500, # in kWh
                "maximum_bandwithd": 2_000.0, # mbps
            },
            electricity_config={"capex": {"solar_panel_costs": 500.0,
                                          "battery_costs": 0.0},
                                "opex": {"cost_per_kwh": 0.1}
                
            },
        )

# Get the school data from a local workspace directory
data_space = ModelDataSpace(global_config.local_workspace_data_space_config) # a number of ways to load this including from project connect APIs

# create and run the mode
model = FiberCostModel(config)
outputs = model.run(data_space) # pass in the model at runtime

assert all([r.technology == 'Fiber' for r in outputs.cost_results])
assert len(outputs.cost_results) == 50 # 50 schools checked
assert outputs.cost_results[0].feasible is False # first school is too far away from a fiber node
assert outputs.cost_results[1].feasible is True

## Sattelite Component Model

In [ ]:
from giga.models.components.satellite_cost_model import SatelliteCostModel
from giga.schemas.conf.models import SatelliteTechnologyCostConf
from giga.data.space.model_data_space import ModelDataSpace


# Specify all the configurations, there are parsers that can help with this in the library
config = SatelliteTechnologyCostConf(
            capex={
                "fixed_costs": 500 # USD hardware installation
            },
            opex={
                "fixed_costs": 0.0, # USD hardware maintance
                "annual_bandwidth_cost_per_mbps": 15.0,
            },
            constraints={
                "maximum_bandwithd": 150.0,  # should be pulled from defaults
                "required_power": 200.0,
            },
            electricity_config={"capex": {"solar_panel_costs": 500.0,
                                          "battery_costs": 0.0},
                                "opex": {"cost_per_kwh": 0.1}
            },
        )

# Get the school data from a local workspace directory
data_space = ModelDataSpace(global_config.local_workspace_data_space_config) # a number of ways to load this including from project connect APIs

# create and run the mode
model = SatelliteCostModel(config)
outputs = model.run(data_space) # pass in the model at runtime

assert all([r.technology == 'Satellite' for r in outputs.cost_results])
assert len(outputs.cost_results) == 50 # 50 schools checked
assert outputs.cost_results[0].capex == 500.0 # Fixed install cost
assert outputs.cost_results[0].opex == 300.0 # Operating cost

## Cellular Component Model

In [ ]:
from giga.models.components.cellular_cost_model import CellularCostModel
from giga.schemas.conf.models import CellularTechnologyCostConf
from giga.data.space.model_data_space import ModelDataSpace


# Specify all the configurations, there are parsers that can help with this in the library
config = CellularTechnologyCostConf(
            capex={"fixed_costs": 500.0},
            opex={
                "fixed_costs": 0.0,
                "annual_bandwidth_cost_per_mbps": 10.0,
            },
            constraints={"maximum_bandwithd": 100.0,
                         "required_power": 10.0,
                         "maximum_range": 8_000}, # in m
            electricity_config={"capex": {"solar_panel_costs": 500.0,
                                          "battery_costs": 0.0},
                                "opex": {"cost_per_kwh": 0.1}
            },
        )

# Get the school data from a local workspace directory
data_space = ModelDataSpace(global_config.local_workspace_data_space_config) # a number of ways to load this including from project connect APIs

# create and run the mode
model = CellularCostModel(config)
outputs = model.run(data_space) # pass in the model at runtime

assert all([r.technology == 'Cellular' for r in outputs.cost_results])
assert len(outputs.cost_results) == 50 # 50 schools checked
assert outputs.cost_results[0].capex == 500.0 # Fixed install cost
assert outputs.cost_results[0].opex == 200.0 # Operating cost